## Subsurface scattering

This example shows how to:
   - setup a glass-like material for subsurface scattering
   - enable light emmision in the volume

![plotoptix ray_tracing_output](https://plotoptix.rnd.team/images/subsurface.jpg "This notebook output")

Glass-like material shader in PlotOptiX can simulate light propagation in a volume with a diffuse scattering. The free path length of the light is set with ``radiation_length``, and the diffusion color is ``subsurface_color``. You can add some color to the surface with ``surface_albedo`` or enable a light emission in the volume to obtain the finest quality materials.

Make some data for a simple scene first:

In [1]:
import numpy as np

rx = (-20, 20)
rz = (-20, 20)
n = 100

x = np.linspace(rx[0], rx[1], n)
z = np.linspace(rz[0], rz[1], n)

X, Z = np.meshgrid(x, z)

# positions of blocks
data = np.stack((X.flatten(), np.full(n*n, -2), Z.flatten())).T
# XZ sizes
size_u = 0.96 * (rx[1] - rx[0]) / (n - 1)
size_w = 0.96 * (rz[1] - rz[0]) / (n - 1)

Setup the raytracer using Tkinter GUI as the output target. Note, it is important to select the **background mode** which supports scattering in volumes: ``AmbientAndVolume``, ``TextureFixed``, or ``TextureEnvironment``.

In [2]:
from plotoptix import TkOptiX

optix = TkOptiX()
optix.set_param(min_accumulation_step=4,      # set more accumulation frames to get rid of the noise
                max_accumulation_frames=512,
                light_shading="Hard")         # use ligth shading best for caustics

optix.set_uint("path_seg_range", 15, 40)      # more path segments to improve simulation in the volume

optix.set_background_mode("AmbientAndVolume") # need one of modes supporting scattering in volumes

Only *diffuse* material is available by default. Other materials need to be configured before using.

In [3]:
from plotoptix.materials import m_clear_glass, m_matt_glass
import copy

m_clear_glass_2 = copy.deepcopy(m_clear_glass)
m_clear_glass_3 = copy.deepcopy(m_clear_glass)
m_clear_glass_4 = copy.deepcopy(m_clear_glass)
m_clear_glass_5 = copy.deepcopy(m_clear_glass)

m_matt_glass_2 = copy.deepcopy(m_matt_glass)

m_light_1 = copy.deepcopy(m_clear_glass)
m_light_2 = copy.deepcopy(m_clear_glass)

optix.setup_material("glass", m_clear_glass)          # clear glass
optix.setup_material("glass_2", m_clear_glass_2)      # diffuse color
optix.setup_material("glass_3", m_clear_glass_3)      # diffuse and albedo color
optix.setup_material("glass_4", m_clear_glass_4)      # diffuse slight
optix.setup_material("glass_5", m_clear_glass_5)      # diffuse slight, textured
optix.setup_material("matt_glass", m_matt_glass)      # matt surface
optix.setup_material("matt_glass_2", m_matt_glass_2)  # matt surface, diffuse volume
optix.setup_material("light_1", m_light_1)            # emissive
optix.setup_material("light_2", m_light_2)            # emissive, textured

Add objects to the scene.

In [4]:
optix.set_data("blocks", pos=data,
               c=0.85 + 0.1*np.random.randint(3, size=data.shape[0]),
               u=[size_u, 0, 0], v=[0, -1, 0], w=[0, 0, size_w],
               geom="Parallelepipeds")

optix.set_data("c_clear", pos=[-3.5, 0, -5], u=[0.25, 0, 0], v=[0, 4, 0], w=[0, 0, 4], c=10, mat="glass", geom="Parallelepipeds")
optix.rotate_geometry("c_clear", [0, 0, -np.pi/4])
optix.set_data("c_diffuse", pos=[-0.5, 0, -5], u=[0.25, 0, 0], v=[0, 4, 0], w=[0, 0, 4], c=10, mat="glass_2", geom="Parallelepipeds")
optix.rotate_geometry("c_diffuse", [0, 0, -np.pi/4])
optix.set_data("c_light", pos=[2.5, 0, -5], u=[0.25, 0, 0], v=[0, 4, 0], w=[0, 0, 4], c=10, mat="light_1", geom="Parallelepipeds")
optix.rotate_geometry("c_light", [0, 0, -np.pi/4])

optix.set_data("s_light_2", pos=[-3.1, 1.5, 1], r=1.5, c=10, mat="light_2", geom="ParticleSetTextured")
optix.set_data("s_diffuse", pos=[0, 1.5, 1], r=1.5, c=10, mat="glass_2")
optix.set_data("s_diffuse_colored", pos=[3.1, 1.5, 1], r=1.5, c=10, mat="glass_3")

optix.set_data("s_matt", pos=[-3.1, 1.5, 4.1], r=1.5, c=10, mat="matt_glass")
optix.set_data("s_diffuse_less", pos=[0, 1.5, 4.1], r=1.5, c=10, mat="glass_4")
optix.set_data("s_diffuse_tex", pos=[3.1, 1.5, 4.1], r=1.5, c=10, mat="glass_5", geom="ParticleSetTextured")

optix.set_data("s_matt_colored", pos=[-3.1, 1.5, 7.2], r=1.5, c=10, mat="matt_glass_2")
optix.set_data("s_light_1", pos=[0, 1.5, 7.2], r=1.5, c=10, mat="light_1")
optix.set_data("s_clear", pos=[3.1, 1.5, 7.2], r=1.5, c=10, mat="glass")

Setup a good point of view, set background and lights.

In [5]:
optix.setup_camera("cam1", cam_type="DoF",
                   eye=[0, 15, 1.55], target=[0, 0, 1.55], up=[1, 0, 0],
                   aperture_radius=0.01, fov=45, focal_scale=0.8)

optix.setup_light("light1", pos=[5, 8, 7], color=[10, 10, 10], radius=1.9)
optix.setup_light("light2", pos=[-6, 8, -5], color=[10, 11, 12], radius=1.3)

exposure = 1.1; gamma = 2.2 
optix.set_float("tonemap_exposure", exposure)
optix.set_float("tonemap_gamma", gamma)
optix.set_float("denoiser_blend", 0.2)
optix.add_postproc("Denoiser")    # apply AI denoiser, or
#optix.add_postproc("Gamma")      # use gamma correction

optix.set_background(0)
optix.set_ambient(0)

Open the GUI.

In [6]:
optix.start()

Modify materials:

In [8]:
m_clear_glass_2["VarFloat"]["radiation_length"] = 0.1                # short w.r.t. the object size
m_clear_glass_2["VarFloat3"]["subsurface_color"] = [ 0.7, 0.85, 1 ]
optix.update_material("glass_2", m_clear_glass_2)

m_clear_glass_3["VarFloat"]["radiation_length"] = 0.1
m_clear_glass_3["VarFloat3"]["subsurface_color"] = [ 0.7, 0.85, 1 ]
m_clear_glass_3["VarFloat3"]["surface_albedo"] = [ 0.6, 0.8, 1 ]     # add some color to reflections
optix.update_material("glass_3", m_clear_glass_3)

m_clear_glass_4["VarFloat"]["radiation_length"] = 1.0                # comparable to the object size
m_clear_glass_4["VarFloat3"]["subsurface_color"] = [ 1, 0.85, 0.7 ]
optix.update_material("glass_4", m_clear_glass_4)

optix.load_texture("rainbow", r"data/rainbow.jpg")
m_clear_glass_5["VarFloat"]["radiation_length"] = 1.0
m_clear_glass_5["VarFloat3"]["subsurface_color"] = [ 1, 0.85, 0.7 ]
m_clear_glass_5["VarFloat3"]["surface_albedo"] = [ 0.9, 1, 1 ]
m_clear_glass_5["ColorTextures"] = [ "rainbow" ]
optix.update_material("glass_5", m_clear_glass_5)

m_matt_glass_2["VarFloat"]["radiation_length"] = 1.0
m_matt_glass_2["VarFloat3"]["subsurface_color"] = [ 1, 0.8, 1 ]
optix.update_material("matt_glass_2", m_matt_glass_2)

m_light_1["VarFloat"]["radiation_length"] = 1.5
m_light_1["VarFloat"]["light_emission"] = 0.1                        # add light on each scattering
m_light_1["VarFloat3"]["subsurface_color"] = [ 0.9, 1, 1 ]           # diffuse and emission base color
optix.update_material("light_1", m_light_1)

optix.load_texture("wood", r"data/wood.jpg")
m_light_2["VarFloat"]["radiation_length"] = 1.5
m_light_2["VarFloat"]["light_emission"] = 0.1
m_light_2["VarFloat3"]["subsurface_color"] = [ 1, 1, 1 ]             # leave the (default) neutral color
m_light_2["ColorTextures"] = [ "wood" ]
optix.update_material("light_2", m_light_2, refresh=True)

Close GUI window, release resources.

In [9]:
optix.close()